In [0]:
from datetime import datetime
import pytz
import pyarrow.parquet as pq
import pandas as pd

from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, to_date, current_timestamp, date_format

In [0]:
try:
    connection_properties = {
        "user": 'cdac',
        "password": dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-secret"),
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    ActiveMutualFundsTable = spark.read \
        .jdbc(url=dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-jdbc"), table='dbo.ActiveMutualFundsNavs', properties=connection_properties)

except Exception as e:
    print(f"Error Loading from SQL Server: {str(e)}")

In [0]:
try:
    connection_properties = {
        "user": 'cdac',
        "password": dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-secret"),
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    KuveraPortfolioExtractsTable = spark.read \
        .jdbc(url=dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-jdbc"), table='dbo.KuveraPortfolioExtracts', properties=connection_properties)
        
    KuveraPortfolioExtractsTable = KuveraPortfolioExtractsTable.drop('nav')
    
except Exception as e:
    print(f"Error writing to SQL Server: {str(e)}")

In [0]:
window_spec = Window.partitionBy("scheme_code").orderBy(col("date").desc())

ActiveMutualFundsTableLatest = (
    ActiveMutualFundsTable
    .withColumn("row_num", row_number().over(window_spec))
    .filter(col("row_num") == 1)
    .drop("row_num")
)

PortolioInformationWithLatestNav = (
    KuveraPortfolioExtractsTable
    .join(ActiveMutualFundsTableLatest, on="scheme_code", how="left")
)

if "insert_date" in PortolioInformationWithLatestNav.columns:
    PortolioInformationWithLatestNav = PortolioInformationWithLatestNav.drop("insert_date")

PortolioInformationWithLatestNav = PortolioInformationWithLatestNav.withColumn("insert_date", date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss"))

In [0]:
try:
    connection_properties = {
        "user": 'cdac',
        "password": dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-secret"),
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    PortolioInformationWithLatestNav.write \
        .mode("overwrite") \
        .jdbc(
            url=dbutils.secrets.get(scope="mutual-fund-keyvalut-secrets", key="storage-database-jdbc"),
            table="Prod.KuveraPortfolioExtractsInformation",
            properties=connection_properties
        )

except Exception as e:
    print(f"Error writing to SQL Server: {str(e)}")